In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1443, with_units=False)
Hist_table


Hours,Entries
IntegerAndSexagesimal,IntegerAndSexagesimal
0 ;,"1 ; 00,57"
0 ;,"2 ; 01,54"
0 ;,"3 ; 02,52"
0 ;,"4 ; 03,49"
0 ;,"5 ; 04,46"
0 ;,"6 ; 05,43"
0 ;,"7 ; 06,41"
0 ;,"8 ; 07,38"
0 ;,"9 ; 08,35"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(30):
    Test_recomp[i][1]=IntegerAndSexagesimal(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=IntegerAndSexagesimal(Sexagesimal('0;0,0'),2)


In [4]:
#  mean motion parameter from the Parisian Alfonsine tables
daily_mean=2*IntegerAndSexagesimal(Sexagesimal("12;11,26,41,37,51,50,39"),7)
REVO=IntegerAndSexagesimal(Sexagesimal("6,0;0,0"),7)


for i in range(30):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
        Test_recomp[i][1]=daily_mean*((i+1)/24)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Hours,Entries
IntegerAndSexagesimal,IntegerAndSexagesimal
0 ;,"1 ; 00,57"
0 ;,"2 ; 01,54"
0 ;,"3 ; 02,52"
0 ;,"4 ; 03,49"
0 ;,"5 ; 04,46"
0 ;,"6 ; 05,43"
0 ;,"7 ; 06,41"
0 ;,"8 ; 07,38"
0 ;,"9 ; 08,35"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-1.3666666666666667, nan)

In [7]:
B.astype(float)

,Entries
Hours,
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0
0 ;,0.0


In [8]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean center hours.csv')
B.to_csv('Lunar mean center hours.zip', index=False, compression=compression_opts)